In [2]:
import os
import subprocess
import pandas as pd
import numpy as np
from pathlib import Path
import datetime
import matplotlib.pyplot as plt

In [2]:
audio_folder = '/Users/caihanzhe/Desktop/Comoros_Data_1'
output_folder = '/Users/caihanzhe/Desktop/Python/Comoros/BirdNet_detection/Outputs'

In [3]:
os.makedirs(output_folder, exist_ok=True)

In [4]:
analyze_script = "/Users/caihanzhe/Desktop/Python/Birdnet_Test/BirdNET-Analyzer/analyze.py"
venv_activation = "/Users/caihanzhe/Desktop/Python/Birdnet_Test/BirdNET-Analyzer/venv-birdnet/bin/activate"

In [ ]:
for subdir, dirs, files in os.walk(audio_folder):
    for file in files:
        if file.endswith('.WAV'):
            audio_path = os.path.join(subdir, file)
            relative_path = os.path.relpath(subdir, audio_folder)
            output_subdir = os.path.join(output_folder, relative_path)
            os.makedirs(output_subdir, exist_ok=True)
            output_path = os.path.join(output_subdir, file.replace('.WAV', '.csv'))

            command = f"source {venv_activation} && python3 {analyze_script} --i \"{audio_path}\" --o \"{output_path}\""
            subprocess.run(command, shell=True)

In [6]:
all_data = pd.DataFrame()

In [7]:
for subdir_1 in os.listdir(output_folder):
    subdir_1_path = os.path.join(output_folder, subdir_1)
    if os.path.isdir(subdir_1_path):
        for file_1 in os.listdir(subdir_1_path):
            if file_1.endswith('.csv'):
                try:
                    path = os.path.join(subdir_1_path, file_1)
                    df = pd.read_csv(path, sep='\t')
                    subdir_1_path = Path(subdir_1_path)
                    site_code, site_name = subdir_1_path.name.split(' - ')
                    date = datetime.datetime.strptime(file_1, '%Y%m%d_%H%M%S.csv')
                    time = date.strftime('%H:%M:%S')
                    df['site_code'] = site_code
                    df['date'] = date
                    df['time'] = time
                    all_data = pd.concat([all_data, df], ignore_index=True)
                except Exception as e:
                    print(f"Failed to read {file_1}: {e}")

In [10]:
all_data

,Selection,View,Channel,start_time,end_time,Low Freq (Hz),High Freq (Hz),Common Name,Species Code,Confidence,Begin Path,File Offset (s),site_code,date,time
0,1,Spectrogram 1,1,51.0,54.0,0,15000,Great Bittern,grebit1,0.4120,/Users/caihanzhe/Desktop/Comoros_Data_1/S18 - ...,51.0,S18,2023-08-09 00:00:01,00:00:01
1,2,Spectrogram 1,1,63.0,66.0,0,15000,Flammulated Owl,flaowl,0.1041,/Users/caihanzhe/Desktop/Comoros_Data_1/S18 - ...,63.0,S18,2023-08-09 00:00:01,00:00:01
2,3,Spectrogram 1,1,66.0,69.0,0,15000,Flammulated Owl,flaowl,0.1241,/Users/caihanzhe/Desktop/Comoros_Data_1/S18 - ...,66.0,S18,2023-08-09 00:00:01,00:00:01
3,4,Spectrogram 1,1,75.0,78.0,0,15000,Flammulated Owl,flaowl,0.1280,/Users/caihanzhe/Desktop/Comoros_Data_1/S18 - ...,75.0,S18,2023-08-09 00:00:01,00:00:01
4,5,Spectrogram 1,1,78.0,81.0,0,15000,Lesser Spotted Woodpecker,leswoo1,0.1011,/Users/caihanzhe/Desktop/Comoros_Data_1/S18 - ...,78.0,S18,2023-08-09 00:00:01,00:00:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76898,190,Spectrogram 1,1,294.0,297.0,0,15000,Great Horned Owl,grhowl,0.2278,/Users/caihanzhe/Desktop/Comoros_Data_1/S19 - ...,294.0,S19,2023-07-16 21:00:01,21:00:01
76899,191,Spectrogram 1,1,297.0,300.0,0,15000,Round-tipped Conehead,t-11879189,0.3771,/Users/caihanzhe/Desktop/Comoros_Data_1/S19 - ...,297.0,S19,2023-07-16 21:00:01,21:00:01
76900,192,Spectrogram 1,1,297.0,300.0,0,15000,Rattler Round-winged Katydid,t-11854789,0.2506,/Users/caihanzhe/Desktop/Comoros_Data_1/S19 - ...,297.0,S19,2023-07-16 21:00:01,21:00:01
76901,193,Spectrogram 1,1,297.0,300.0,0,15000,Protean Shieldback,t-11844441,0.1933,/Users/caihanzhe/Desktop/Comoros_Data_1/S19 - ...,297.0,S19,2023-07-16 21:00:01,21:00:01


In [9]:
all_data.rename(columns={'Begin Time (s)': 'start_time', 'End Time (s)': 'end_time'}, inplace=True)

In [11]:
all_data.to_csv('/Users/caihanzhe/Desktop/Python/Comoros/BirdNet_detection/all_data.csv', index=False)

In [7]:
duplicate_keys = all_data[all_data.duplicated(['start_time', 'end_time', 'site_code', 'date'], keep=False)]
print(duplicate_keys)


       Selection           View  Channel  start_time  end_time  Low Freq (Hz)  \
17            18  Spectrogram 1        1       207.0     210.0              0   
18            19  Spectrogram 1        1       207.0     210.0              0   
20            21  Spectrogram 1        1       237.0     240.0              0   
21            22  Spectrogram 1        1       237.0     240.0              0   
34             7  Spectrogram 1        1       135.0     138.0              0   
...          ...            ...      ...         ...       ...            ...   
76898        190  Spectrogram 1        1       294.0     297.0              0   
76899        191  Spectrogram 1        1       297.0     300.0              0   
76900        192  Spectrogram 1        1       297.0     300.0              0   
76901        193  Spectrogram 1        1       297.0     300.0              0   
76902        194  Spectrogram 1        1       297.0     300.0              0   

       High Freq (Hz)      

In [3]:
all_data = pd.read_csv('/Users/caihanzhe/Desktop/Python/Comoros/BirdNet_detection/all_data.csv')

In [4]:
all_data

,Selection,View,Channel,start_time,end_time,Low Freq (Hz),High Freq (Hz),Common Name,Species Code,Confidence,Begin Path,File Offset (s),site_code,date,time
0,1,Spectrogram 1,1,51.0,54.0,0,15000,Great Bittern,grebit1,0.4120,/Users/caihanzhe/Desktop/Comoros_Data_1/S18 - ...,51.0,S18,2023-08-09 00:00:01,00:00:01
1,2,Spectrogram 1,1,63.0,66.0,0,15000,Flammulated Owl,flaowl,0.1041,/Users/caihanzhe/Desktop/Comoros_Data_1/S18 - ...,63.0,S18,2023-08-09 00:00:01,00:00:01
2,3,Spectrogram 1,1,66.0,69.0,0,15000,Flammulated Owl,flaowl,0.1241,/Users/caihanzhe/Desktop/Comoros_Data_1/S18 - ...,66.0,S18,2023-08-09 00:00:01,00:00:01
3,4,Spectrogram 1,1,75.0,78.0,0,15000,Flammulated Owl,flaowl,0.1280,/Users/caihanzhe/Desktop/Comoros_Data_1/S18 - ...,75.0,S18,2023-08-09 00:00:01,00:00:01
4,5,Spectrogram 1,1,78.0,81.0,0,15000,Lesser Spotted Woodpecker,leswoo1,0.1011,/Users/caihanzhe/Desktop/Comoros_Data_1/S18 - ...,78.0,S18,2023-08-09 00:00:01,00:00:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76898,190,Spectrogram 1,1,294.0,297.0,0,15000,Great Horned Owl,grhowl,0.2278,/Users/caihanzhe/Desktop/Comoros_Data_1/S19 - ...,294.0,S19,2023-07-16 21:00:01,21:00:01
76899,191,Spectrogram 1,1,297.0,300.0,0,15000,Round-tipped Conehead,t-11879189,0.3771,/Users/caihanzhe/Desktop/Comoros_Data_1/S19 - ...,297.0,S19,2023-07-16 21:00:01,21:00:01
76900,192,Spectrogram 1,1,297.0,300.0,0,15000,Rattler Round-winged Katydid,t-11854789,0.2506,/Users/caihanzhe/Desktop/Comoros_Data_1/S19 - ...,297.0,S19,2023-07-16 21:00:01,21:00:01
76901,193,Spectrogram 1,1,297.0,300.0,0,15000,Protean Shieldback,t-11844441,0.1933,/Users/caihanzhe/Desktop/Comoros_Data_1/S19 - ...,297.0,S19,2023-07-16 21:00:01,21:00:01


In [6]:
species_counts = all_data['Common Name'].value_counts()
print(species_counts)


Common Name
Lesser Spotted Woodpecker     21997
Akohekohe                      6125
Rock Wren                      5538
Great Horned Owl               3995
Round-tipped Conehead          3424
                              ...  
Splendid Sunbird                  1
Lineated Foliage-gleaner          1
Broad-billed Hummingbird          1
Zeledon's Antbird                 1
Indian Paradise-Flycatcher        1
Name: count, Length: 1644, dtype: int64
